## 데이터 파악 (훈련데이터,테스트 데이터 브랜드명 추출)
### 추출시 공통된 브랜드가없다.

In [3]:
import os
import json

def extract_brand_names_from_first_item(directory_path):
    brand_names = []

    for filename in os.listdir(directory_path):
        if filename.endswith('.json'):
            file_path = os.path.join(directory_path, filename)
            try:
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)

                    if isinstance(data, list) and len(data) > 0:
                        first_item = data[0]
                        brand_name = first_item.get("JNG_INFO", {}).get("BRAND_NM")
                        brand_num = first_item.get("JNG_INFO", {}).get("JNG_IFRMP_SN")
                        if brand_name:
                            brand_names.append((filename, brand_name))
                        else:
                            print(f"[경고] '{filename}'에서 BRAND_NM을 찾을 수 없습니다.")
                    else:
                        print(f"[경고] '{filename}'은 리스트가 아니거나 비어 있습니다.")

            except Exception as e:
                print(f"[에러] '{filename}' 처리 중 오류 발생: {e}")

    return brand_names

def save_brand_names_to_txt(brand_list, output_path):
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            for filename, brand in brand_list:
                f.write(f"{filename}: {brand}\n")
        print(f"[저장 완료] {output_path}")
    except Exception as e:
        print(f"[에러] 파일 저장 중 오류 발생: {e}")


# 사용 예시
if __name__ == "__main__":
    train_directory = ("/home/sm7540/workspace/franchise_rag/data/train")
    test_directory = ("/home/sm7540/workspace/franchise_rag/data/test")
    train_brand_nm =  extract_brand_names_from_first_item(train_directory)
    test_brand_nm =  extract_brand_names_from_first_item(test_directory)
    # 텍스트 파일로 저장
    save_brand_names_to_txt(train_brand_nm, "train_brands.txt")
    save_brand_names_to_txt(test_brand_nm, "test_brands.txt")
    # 브랜드명만 추출해서 set으로 변환
    train_brands = set([brand for _, brand in train_brand_nm])
    test_brands = set([brand for _, brand in test_brand_nm])

    # 공통 브랜드명 추출
    common_brands = train_brands.intersection(test_brands)

    # 결과 출력
    print("\n공통된 브랜드명:")
    for brand in sorted(common_brands):
        print(f"- {brand}")



[저장 완료] train_brands.txt
[저장 완료] test_brands.txt

공통된 브랜드명:


### 사전지식베이스 구축 [fewshot 용 QA쌍 지식베이스]

In [ ]:
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.docstore.document import Document
from langchain_chroma import Chroma
from config import Settings
import json
import os

settings = Settings(JSON_PATH="/home/sm7540/workspace/franchise_rag/data/train/1056757501.json")

# HuggingFace 임베딩 모델 초기화
try:
    embeddings = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_PATH,
        model_kwargs={'device': 'cuda'}
    )
    print("로컬 임베딩 모델 로드 성공")
except Exception:
    print("로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다")
    embeddings = HuggingFaceEmbeddings(
        model_name=settings.EMBEDDING_MODEL_NAME,
        model_kwargs={'device': 'cuda'}
    )

print(f"임베딩 모델 {settings.EMBEDDING_MODEL_NAME} 로딩 완료")

# 벡터 저장소 디렉토리 생성
vector_db_path = settings.VECTOR_DB_PATH
os.makedirs(vector_db_path, exist_ok=True)

# JSON 데이터 로드
try:
    with open(settings.JSON_PATH, 'r', encoding='utf-8') as file:
        contracts_data = json.load(file)
    print(f"총 {len(contracts_data)} 개의 계약서 데이터를 로드했습니다.")
except FileNotFoundError:
    print(f"파일을 찾을 수 없습니다: {settings.JSON_PATH}")
    contracts_data = []
    exit()
except json.JSONDecodeError:
    print("JSON 파일 파싱 중 오류가 발생했습니다.")
    contracts_data = []
    exit()

# 문서 객체 생성
documents = []
ids = []

for contract_idx, contract in enumerate(contracts_data):
    summary_text = contract.get("QL", {}).get("ABSTRACTED_SUMMARY_TEXT", "").strip()
    qas = contract.get("QL", {}).get("QAs", [])

    # 메타데이터 구성
    metadata = {
        "CHNK_NO": contract["CHNK_NO"],
        "SMRT_CHNK_NO": contract["SMRT_CHNK_NO"],
        "JNG_BIZ_CRTRA_YR": contract["JNG_INFO"]["JNG_BIZ_CRTRA_YR"],
        "JNGHDQRTRS_CONM_NM": contract["JNG_INFO"]["JNGHDQRTRS_CONM_NM"],
        "BRAND_NM": contract["JNG_INFO"]["BRAND_NM"],
        "JNG_IFRMP_SN": contract["JNG_INFO"]["JNG_IFRMP_SN"],
        "ATTRB_MNNO": contract["ATTRB_INFO"]["ATTRB_MNNO"],
        "KORN_ATTRB_NM": contract["ATTRB_INFO"]["KORN_ATTRB_NM"],
        "UP_ATTRB_MNNO": contract["ATTRB_INFO"]["UP_ATTRB_MNNO"],
        "KORN_UP_ATRB_NM": contract["ATTRB_INFO"]["KORN_UP_ATRB_NM"],
        "source": f"{contract['JNG_INFO']['JNG_IFRMP_SN']}.json",
        "QAs": json.dumps(qas, ensure_ascii=False)  # 문자열로 저장  # QA 전체 포함
    }
    
    doc_id = f"{contract['JNG_INFO']['JNG_IFRMP_SN']}_{contract_idx}"
    documents.append(Document(page_content=summary_text, metadata=metadata))
    ids.append(doc_id)

print(f"{len(documents)}개의 요약 기반 문서 객체 생성 완료")

# Chroma 벡터 스토어 생성
try:
    vector_store = Chroma.from_documents(
        documents=documents,
        embedding=embeddings,
        collection_name="contracts_summary_collection",
        persist_directory=vector_db_path,
        ids=ids
    )
    print(f"✅ 벡터 스토어 생성 완료. 저장 경로: {vector_db_path}")
except Exception as e:
    print(f"❌ 벡터 스토어 생성 중 오류 발생: {str(e)}")


로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다
임베딩 모델 nlpai-lab/KURE-v1 로딩 완료
총 57 개의 계약서 데이터를 로드했습니다.
57개의 요약 기반 문서 객체 생성 완료
✅ 벡터 스토어 생성 완료. 저장 경로: ./vector_db/franchise


## 추론

In [ ]:
from config import Settings
from fewshot_franchise import GeminiFewShotFranchiseService

settings = Settings(DEVICE="cuda")

rag_service = GeminiFewShotFranchiseService(
    api_key=settings.GEMINI_API_KEY,
    config={
        "vector_db_path": settings.VECTOR_DB_PATH,
        "model_name": settings.MODEL_NAME,
        "embedding_model_path": settings.EMBEDDING_MODEL_PATH,
        "device":settings.DEVICE,
        "vectorstore_search_k":1
    }
)

response = rag_service.inference("서영에프앤비의 주소는 어디인가요?")



/home/sm7540/workspace/franchise_rag/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-05-11 20:46:02,510 - franchise - INFO - 로컬 임베딩 모델 로드 중: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 20:46:04,248 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9
2025-05-11 20:46:04,249 - franchise - ERROR - 로컬 임베딩 모델 로드 실패, 온라인 모델을 사용합니다: Path C:\Users\oreo\.cache\huggingface\hub\models--nlpai-lab--KURE-v1\snapshots\d14c8a9423946e268a0c9952fecf3a7aabd73bd9 not found
2025-05-11 20:46:04,249 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransforme

======사용하는 프롬프트는======
당신은 정보공개서를 기반으로 가맹본부에 대한 질문에 정확하고 사실 기반의 답변을 제공하는 AI 어시스턴트입니다.

다음은 예시 질문과 답변입니다:
Q: 초대창의 일반상권 영업지역 설정 기준은 무엇인가요?
A: 초대창의 일반상권 영업지역은 가맹점의 출입문을 기준으로 도보 반경 500m 이내로 설정됩니다. 단, 영업지역 내 특수상권은 별도의 영업지역으로 간주되어 해당되지 않습니다.

Q: 특수상권의 영업지역 설정 방법은 어떻게 되나요?
A: 초대창의 특수상권 영업지역은 해당 특수상권 내로 설정됩니다.

Q: 초대창의 특급상권 영업지역은 어떻게 설정되나요?
A: 초대창의 특급상권 영업지역은 매장 주 출입구를 기준으로 도보 반경 200m 이내로 설정됩니다. 또한 영업지역 내 특수상권은 별도의 영업지역으로 간주되어 포함되지 않습니다.

[참고 문서]
가맹본부: (주)미지에듀
 브랜드: SBS아카데미뷰티스쿨
 목차: 2) 영업지역 설정기준
당사는 영업지역을 다음과 같이 정하고 있습니다.<table><tr><td>설정 기준 </td><td>설정방법 </td></tr><tr><td>서울시 각 구 1지점 </td><td>- 설정기준에 의해 가맹희망자와 협의 후 설정 - 별지에 영업지역 표시(지도별첨을 원칙으로 함) </td></tr><tr><td>광역시 2지점 </td></tr><tr><td>시 단위 1지점 </td></tr></table>

[질문]
서영에프앤비의 주소는 어디인가요?

[답변]

